In [1]:
# import reddit api wrapper
import praw
import pandas as pd
import numpy as np

import datetime
import tldextract

pd.options.display.float_format = '{:.2f}'.format

# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /ANLY560_website
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload


/Users/prcork/Georgetown/Spring2023/TimeSeries/ANLY560_website


In [2]:
%autoreload
from _functions.get_reddit_data import get_submissions, get_comments

Version 7.6.0 of praw is outdated. Version 7.6.1 was released Friday November 11, 2022.


In [3]:
# get 1000 highest upvoted submissions from the last month
magicTCG = get_submissions(subreddit="magicTCG", time_filter="month", limit=1000)
finance = get_submissions(subreddit="mtgfinance", time_filter="month", limit=1000)

In [ ]:
# print stats to include in summary tables

print("r/magicTCG")
print(magicTCG.shape)
print(magicTCG.ups.mean())
print(magicTCG.num_comments.mean())
print()
print("r/finance")
print(finance.shape)
print(finance.ups.mean())
print(finance.num_comments.mean())


In [ ]:
# categorize and merge subreddits for preprocessing

magicTCG['subreddit'] = 'magicTCG'
finance['subreddit'] = 'mtgfinance'

df = pd.concat([magicTCG, finance])

In [ ]:
# convert UTC to datetime, then store only date
df['date'] = df['created_utc'].apply(lambda utc_entry: datetime.datetime.utcfromtimestamp(utc_entry))
df['date'] = df['date'].dt.date

# if the last four characters of the url are an image filetype, categorize the submission as an image
df['image'] = np.where(df.url.str[-4:].isin(['.jpg', '.png', '.gif']), 1, 0)

# split the url into sub, domain, and extension, store only domain
df['domain'] = df['url'].apply(lambda x: tldextract.extract(x).domain)

print(df.shape)
df.sample(5)

In [ ]:
df.drop(['created_utc', 'url'], axis=1, inplace=True)
df.to_csv("_data/reddit_top_month_submissions.csv", index=False)